In [12]:
from pathlib import Path
import fitz
import re

In [28]:
path = Path('../../ocr-projects/projects/buzzi/data/bmjopen-2016-011025_Proof_hi.pdf')
page_range = range(23, 64)
out_file = "bmjopen-2016-011025_Proof_hi.pdf-refs.pdf"

assert path.exists()
doc = fitz.open(path)

omit_pattern = re.compile(r"for peer review only", re.IGNORECASE)
omit_pattern2 = re.compile(r"bmj open", re.IGNORECASE)
omit_pattern3 = re.compile(r"page\s+\d+\s+of\s+\d+", re.IGNORECASE)
omit_pattern4 = re.compile(r"^\s*\d+\s*$", re.MULTILINE)


doc2 = fitz.Document()
for i in page_range:
    page = doc[i]
    new_page = doc2.new_page(
        -1, width=page.rect.width, height=page.rect.height,
    )
    page_dict = page.get_text("dict", sort=False)
    for block in page_dict['blocks'][:-1]:
        if "lines" not in block:
            continue
        for line in block['lines']:
            for span in line['spans']:
                bbox = span['bbox']
                size = span['size']
                font = span['font']
                text = span['text']
                match = omit_pattern.search(text)
                if match:
                    continue
                match = omit_pattern2.search(text)
                if match:
                    continue
                match = omit_pattern3.search(text)
                if match:
                    continue
                match = omit_pattern4.search(text)
                if match:
                    continue
                # print(f"writing {text} at {bbox} with {size} and {font}")
                res = new_page.insert_textbox(
                    fitz.Rect(bbox[0], bbox[1], bbox[2], bbox[3]), 
                    text, 
                    fontsize=size, 
                    fontname="times-roman"
                )
                if res < 0:
                    res = new_page.insert_textbox(
                        fitz.Rect(bbox[0], bbox[1], bbox[2]-res, bbox[3]-res), 
                        text, 
                        fontsize=size, 
                        fontname="times-roman"
                    )

doc2.save(out_file)